In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from datetime import date, timedelta
import datetime as dt

In [234]:
cluster7=pd.read_csv('clust_7.csv') #cluster numbers
cluster12=pd.read_csv('clust_12.csv')

In [235]:
data=pd.read_csv('integrated_data.csv')

In [236]:
train_df=data[data.month !=1] 
test_df=data[data.month ==1 ] 
test_df=test_df.reset_index(drop=True)

In [237]:
train_df.shape

(1166984, 35)

In [238]:
data_cluster=pd.read_csv('data_clustering_R') #dataset used for clustering

In [239]:
data_cluster['clust_7']=cluster7['x'] #joining the clusters

In [240]:
data_cluster=data_cluster[['key','clust_7']]

In [241]:
#outputting it as csv
data_cluster.to_csv('clusters_key_7') 

In [242]:
#outputting it as csv
data_cluster1=data_cluster.copy()
data_cluster1['clust_12']=cluster12['x']
data_cluster1=data_cluster1[['key','clust_12']]
data_cluster1.to_csv('clusters_key_12')

In [243]:
data_cluster.head()

,key,clust_7
0,10000 XL ( 158-170 ),1
1,10001 L,1
2,10003 3 (35-38 ),2
3,10003 4 ( 39-42 ),1
4,10003 5 ( 43-46 ),1


In [244]:
train_df.head()

,Unnamed: 0,key,weekday,day,month,date,rrp,new size_L,new size_M,new size_S,...,new size_44,new size_43,units,avg_temp,median_temp,company_offer,holiday,sum_unit,price_daily_change,new_product
0,0,19671 39 1/3,6,1,10,2017-10-01,190.43,0,0,0,...,0,0,0.0,12.5625,12.50,0,0,0.0,0.0,0
1,1,19671 39 1/3,0,2,10,2017-10-02,190.43,0,0,0,...,0,0,0.0,13.3125,13.75,0,0,0.0,0.0,0
2,2,19671 39 1/3,1,3,10,2017-10-03,190.43,0,0,0,...,0,0,0.0,12.1875,12.50,0,1,0.0,0.0,0
3,3,19671 39 1/3,2,4,10,2017-10-04,190.43,0,0,0,...,0,0,0.0,10.7500,10.75,0,0,0.0,0.0,0
4,4,19671 39 1/3,3,5,10,2017-10-05,190.43,0,0,0,...,0,0,1.0,11.7500,11.50,0,0,1.0,0.0,0


The below code has been taken from the following adress: https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

In [245]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [246]:
d1 = date(2017, 10, 1)  # start date
d2 = date(2017, 12, 31)  # end date
d3=date(2018, 1, 31)

delta = d2 - d1         # timedelta

pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)

post_day=[] 
delta_2 = d3 - d2

for i in range(delta_2.days + 1):
    add=str(d2 + timedelta(days=i))
    post_day.append(add)

In [247]:
df_days_pre = pd.DataFrame({'date':pre_day})
df_days_post = pd.DataFrame({'date':post_day})

In [248]:
len(pre_day)

92

In [249]:
train=train_df[['date','key','units']]

In [250]:
train.shape

(1166984, 3)

# 1. Windowing with clusters (Lag=1)

### Data Preparation:

In [19]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [20]:
sales_prev=train.loc[train['date']=='2017-12-31'].sort_values('date')[['key','units']]
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-31'})

In [21]:
data_frame_2.head()

,key,2017-12-31
0,19671 39 1/3,0.0
1,19671 40,0.0
2,19671 41 1/3,1.0
3,19671 42,0.0
4,19671 42 2/3,0.0


In [22]:
data_frame=pd.merge(data_frame,data_frame_2,on='key')

In [23]:
data_frame.head()

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2018-01-24,2018-01-25,2018-01-26,2018-01-27,2018-01-28,2018-01-29,2018-01-30,2018-01-31,key,2017-12-31
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19671 39 1/3,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19671 40,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19671 41 1/3,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19671 42,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19671 42 2/3,0.0


In [24]:
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner')

In [25]:
c_dataframe.head()

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2018-01-25,2018-01-26,2018-01-27,2018-01-28,2018-01-29,2018-01-30,2018-01-31,key,2017-12-31,clust_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19671 39 1/3,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19671 40,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19671 41 1/3,1.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19671 42,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19671 42 2/3,0.0,2


In [26]:
c_train=train.merge(data_cluster,on='key',how='inner')

In [27]:
c_train.head()

,date,key,units,clust_7
0,2017-10-01,19671 39 1/3,0.0,1
1,2017-10-02,19671 39 1/3,0.0,1
2,2017-10-03,19671 39 1/3,0.0,1
3,2017-10-04,19671 39 1/3,0.0,1
4,2017-10-05,19671 39 1/3,1.0,1


In [28]:
## Running the clusters
def days_1_cluster(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    print(c_train_c1.shape)
    print(c_train_c1.key.nunique())
    train_comb_pivot=c_train_c1.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0) #assign zero sales
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=1, n_out=1, dropnan=True)
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose()
    model.fit(train_cluster1[['var1(t-1)']],train_cluster1[['var1(t)']])

    days=range(1,32)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=1)
        pred_date_before=predicted_date- timedelta(days=1)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=cluster1_frame[bet_day]
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    return(cluster1_frame)

In [30]:
regr = linear_model.LinearRegression()
cluster1_frame=days_1_cluster(c_dataframe,c_train,1,regr)

(806078, 3)
8858


In [31]:
regr = linear_model.LinearRegression()
cluster2_frame=days_1_cluster(c_dataframe,c_train,2,regr)

(156520, 3)
1720


In [32]:
regr = linear_model.LinearRegression()
cluster3_frame=days_1_cluster(c_dataframe,c_train,3,regr)

(25844, 3)
284


In [33]:
regr = linear_model.LinearRegression()
cluster4_frame=days_1_cluster(c_dataframe,c_train,4,regr)

(1001, 3)
11


In [34]:
regr = linear_model.LinearRegression()
cluster5_frame=days_1_cluster(c_dataframe,c_train,5,regr)

(8463, 3)
93


In [35]:
regr = linear_model.LinearRegression()
cluster6_frame=days_1_cluster(c_dataframe,c_train,6,regr)

(3640, 3)
40


In [36]:
regr = linear_model.LinearRegression()
cluster7_frame=days_1_cluster(c_dataframe,c_train,7,regr)

(364, 3)
4


### Data Measurement:

In [37]:
# performance Measurement
import math
def cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,date_ass):
    test_data['pid']=test_data["pid"].astype(int)
    test_data['key']=test_data["pid"].astype(str) +" "+ test_data["size"].astype(str) 
    test_data=test_data[['key','stock','sold_out_date']]
    d1 = date(2018, 1, 1)  
    d2 = date(2018, 1, 31)  
    delta = d2 - d1         
    pre_day=[]
    for i in range(delta.days + 1):
        add=str(d1 + timedelta(days=i))
        pre_day.append(add)

    all_=test_data.key.unique()
    n=len(test_data.key.unique())
    i=0
    k=0
    n1=len(pre_day)
    days=[]
    ids=[]
    while(i<n):
        b=all_[i]
        k=0
        while(k<n1):
            a=pre_day[k]
            ids.append(b)
            days.append(a)
            k=k+1
        i=i+1
    test_days = pd.DataFrame({'date':days,'key':ids})
    test_comb=test_days.merge(test_data,on=['key'],how='outer')
    test_comb=test_comb.fillna(0)
    c_test=test_comb.merge(data_cluster,on='key',how='left')
    c_test['clust_7']=c_test['clust_7'].fillna(8)
    check=c_test[['key','sold_out_date','clust_7']]
    check=check.drop_duplicates()
    check['sol_out_pred']=0
    i=0
    all_=c_test.key.unique()
    sold_out=[]
    n=len(all_)
    while(i<n):
        subset_w=c_test.loc[c_test['key']==all_[i]]
        check_w=check.loc[check['key']==all_[i]]
        stock=subset_w['stock'].values[0]
        clust=subset_w['clust_7'].values[0]
        n1=len(pre_day)
        k=0
        while(k<n1): #day based
            day=pre_day[k]
            that_day_sale=0
            if(clust==1):
                that_day_sale=cluster1_frame[day].values[0]
            if(clust==2):
                that_day_sale=cluster2_frame[day].values[0]  
            if(clust==3):
                that_day_sale=cluster3_frame[day].values[0]
            if(clust==4):
                that_day_sale=cluster4_frame[day].values[0]
            if(clust==5):
                that_day_sale=cluster5_frame[day].values[0]
            if(clust==6):
                that_day_sale=cluster6_frame[day].values[0]
            if(clust==7):
                that_day_sale=cluster7_frame[day].values[0]
            if(clust==8):
                sold_out.append(date_ass)
                break
            stock=stock-that_day_sale
            if((stock==0) | (stock<0) ):
                sold_out.append(day)
                break
            if(k==(n1-1)):
                sold_out.append(date_ass)
            k=k+1
        i=i+1
        
    check['sol_out_pred']=sold_out
    check["date_sold_out_date"] = pd.to_datetime(check["sold_out_date"],format="%Y/%m/%d")
    check["date_sold_out_pred"] = pd.to_datetime(check["sol_out_pred"],format="%Y/%m/%d")
    days_differ=check["date_sold_out_date"]-check["date_sold_out_pred"]
    
    check['days_differ']=days_differ
    check['correct_differ']=abs(check['days_differ'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days))

    cluster_size=check.clust_7.value_counts()
    by_group=check.groupby(['clust_7']).sum()
    by_group['clust_size']=cluster_size
    by_group['mean']=by_group['correct_differ']/by_group['clust_size']
    print(by_group)
    
    
    days=pd.DataFrame(days_differ,columns=['days_differ'])
    days['correct_differ']=abs(days['days_differ'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days))
    summ=sum(days['correct_differ'])
    return(math.sqrt(summ))

In [38]:
test_data=pd.read_csv('test_0.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39175        4668  8.392245
2.0               12598        1575  7.998730
3.0                1795         281  6.387900
4.0                  55          11  5.000000
5.0                 539          93  5.795699
6.0                 232          40  5.800000
7.0                   7           4  1.750000
8.0               10322        1470  7.021769


254.4071539874616

In [39]:
test_data=pd.read_csv('test_1.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39542        4668  8.470865
2.0               12770        1575  8.107937
3.0                1785         281  6.352313
4.0                  58          11  5.272727
5.0                 478          93  5.139785
6.0                 210          40  5.250000
7.0                  16           4  4.000000
8.0               10299        1470  7.006122


255.26065110000798

In [40]:
test_data=pd.read_csv('test_2.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39418        4668  8.444302
2.0               12723        1575  8.078095
3.0                1842         281  6.555160
4.0                  57          11  5.181818
5.0                 508          93  5.462366
6.0                 196          40  4.900000
7.0                  17           4  4.250000
8.0               10273        1470  6.988435


255.01764644824092

In [41]:
test_data=pd.read_csv('test_3.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39333        4668  8.426093
2.0               12824        1575  8.142222
3.0                1788         281  6.362989
4.0                  62          11  5.636364
5.0                 467          93  5.021505
6.0                 230          40  5.750000
7.0                  11           4  2.750000
8.0               10214        1470  6.948299


254.8116951790086

In [42]:
test_data=pd.read_csv('test_4.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39526        4668  8.467438
2.0               12859        1575  8.164444
3.0                1753         281  6.238434
4.0                  57          11  5.181818
5.0                 534          93  5.741935
6.0                 211          40  5.275000
7.0                  18           4  4.500000
8.0               10187        1470  6.929932


255.23518566216532

# 2. Windowing with clusters (Lag=5)

In [43]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [44]:
sales_prev=train.loc[train['date']=='2017-12-31'].sort_values('date')[['key','units']]
sales_prev1=train.loc[train['date']=='2017-12-30'].sort_values('date')[['key','units']]
sales_prev2=train.loc[train['date']=='2017-12-29'].sort_values('date')[['key','units']]
sales_prev3=train.loc[train['date']=='2017-12-28'].sort_values('date')[['key','units']]
sales_prev4=train.loc[train['date']=='2017-12-27'].sort_values('date')[['key','units']]
sales_prev5=train.loc[train['date']=='2017-12-26'].sort_values('date')[['key','units']]

In [45]:
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-26'})

In [46]:
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')

In [47]:
data_frame.head()

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2018-01-29,2018-01-30,2018-01-31,key,2017-12-31,2017-12-30,2017-12-29,2017-12-28,2017-12-27,2017-12-26
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,19671 39 1/3,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,19671 40,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,19671 41 1/3,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,19671 42,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,19671 42 2/3,0.0,0.0,0.0,1.0,0.0,0.0


In [48]:
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner')

In [49]:
c_dataframe.head()

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2018-01-30,2018-01-31,key,2017-12-31,2017-12-30,2017-12-29,2017-12-28,2017-12-27,2017-12-26,clust_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19671 39 1/3,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19671 40,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19671 41 1/3,1.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19671 42,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19671 42 2/3,0.0,0.0,0.0,1.0,0.0,0.0,2


In [50]:
c_train=train.merge(data_cluster,on='key',how='inner')

In [51]:
## Running the clusters
def days_5_cluster(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    print(c_train_c1.shape)
    print(c_train_c1.key.nunique())
    train_comb_pivot=c_train_c1.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0) #assign zero sales
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=5, n_out=1, dropnan=True)
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose()
    model.fit(train_cluster1[['var1(t-5)','var1(t-4)','var1(t-3)','var1(t-2)','var1(t-1)']],train_cluster1[['var1(t)']])

    days=range(1,32)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=1)
        pred_date_before=predicted_date- timedelta(days=5)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=cluster1_frame[bet_day]
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    return(cluster1_frame)

In [52]:
c_dataframe.head()

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2018-01-30,2018-01-31,key,2017-12-31,2017-12-30,2017-12-29,2017-12-28,2017-12-27,2017-12-26,clust_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19671 39 1/3,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19671 40,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19671 41 1/3,1.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19671 42,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19671 42 2/3,0.0,0.0,0.0,1.0,0.0,0.0,2


In [53]:
c_train.head()

,date,key,units,clust_7
0,2017-10-01,19671 39 1/3,0.0,1
1,2017-10-02,19671 39 1/3,0.0,1
2,2017-10-03,19671 39 1/3,0.0,1
3,2017-10-04,19671 39 1/3,0.0,1
4,2017-10-05,19671 39 1/3,1.0,1


In [54]:
regr = linear_model.LinearRegression()
cluster1_frame=days_5_cluster(c_dataframe,c_train,1,regr)

(806078, 3)
8858


In [55]:
regr = linear_model.LinearRegression()
cluster2_frame=days_5_cluster(c_dataframe,c_train,2,regr)

(156520, 3)
1720


In [56]:
regr = linear_model.LinearRegression()
cluster3_frame=days_5_cluster(c_dataframe,c_train,3,regr)

(25844, 3)
284


In [57]:
regr = linear_model.LinearRegression()
cluster4_frame=days_5_cluster(c_dataframe,c_train,4,regr)

(1001, 3)
11


In [58]:
regr = linear_model.LinearRegression()
cluster5_frame=days_5_cluster(c_dataframe,c_train,5,regr)

(8463, 3)
93


In [59]:
regr = linear_model.LinearRegression()
cluster6_frame=days_5_cluster(c_dataframe,c_train,6,regr)

(3640, 3)
40


In [60]:
regr = linear_model.LinearRegression()
cluster7_frame=days_5_cluster(c_dataframe,c_train,7,regr)

(364, 3)
4


## Performance Measure

In [61]:
test_data=pd.read_csv('test_0.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               40188        4668  8.609254
2.0               12600        1575  8.000000
3.0                1783         281  6.345196
4.0                  55          11  5.000000
5.0                 540          93  5.806452
6.0                 230          40  5.750000
7.0                   7           4  1.750000
8.0               10322        1470  7.021769


256.3688748658854

In [62]:
test_data=pd.read_csv('test_1.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               40568        4668  8.690660
2.0               12770        1575  8.107937
3.0                1782         281  6.341637
4.0                  58          11  5.272727
5.0                 477          93  5.129032
6.0                 210          40  5.250000
7.0                  15           4  3.750000
8.0               10299        1470  7.006122


257.25279395956034

In [63]:
test_data=pd.read_csv('test_2.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               40414        4668  8.657669
2.0               12718        1575  8.074921
3.0                1835         281  6.530249
4.0                  58          11  5.272727
5.0                 507          93  5.451613
6.0                 196          40  4.900000
7.0                  17           4  4.250000
8.0               10273        1470  6.988435


256.9396816375392

In [64]:
test_data=pd.read_csv('test_3.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               40366        4668  8.647386
2.0               12822        1575  8.140952
3.0                1780         281  6.334520
4.0                  62          11  5.636364
5.0                 467          93  5.021505
6.0                 230          40  5.750000
7.0                  11           4  2.750000
8.0               10214        1470  6.948299


256.81121470839236

In [65]:
test_data=pd.read_csv('test_4.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               40557        4668  8.688303
2.0               12853        1575  8.160635
3.0                1755         281  6.245552
4.0                  56          11  5.090909
5.0                 535          93  5.752688
6.0                 210          40  5.250000
7.0                  18           4  4.500000
8.0               10187        1470  6.929932


257.2372445817285

# 3. Windowing with clusters (Lag=10)

In [68]:
cluster7=pd.read_csv('clust_7.csv') #cluster numbers
data_cluster=pd.read_csv('data_clustering_R') #dataset used for clustering
data_cluster['clust_7']=cluster7['x'] #joining the clusters
data_cluster=data_cluster[['key','clust_7']]
d1 = date(2017, 10, 1)  # start date
d2 = date(2017, 12, 31)  # end date
d3=date(2018, 1, 31)

delta = d2 - d1         # timedelta

pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)

post_day=[] 
delta_2 = d3 - d2

for i in range(delta_2.days + 1):
    add=str(d2 + timedelta(days=i))
    post_day.append(add)
df_days_pre = pd.DataFrame({'date':pre_day})
df_days_post = pd.DataFrame({'date':post_day})
train=train_df[['date','key','units']]

In [69]:
train.shape

(1166984, 3)

In [70]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [71]:
sales_prev=train.loc[train['date']=='2017-12-31'].sort_values('date')[['key','units']]
sales_prev1=train.loc[train['date']=='2017-12-30'].sort_values('date')[['key','units']]
sales_prev2=train.loc[train['date']=='2017-12-29'].sort_values('date')[['key','units']]
sales_prev3=train.loc[train['date']=='2017-12-28'].sort_values('date')[['key','units']]
sales_prev4=train.loc[train['date']=='2017-12-27'].sort_values('date')[['key','units']]
sales_prev5=train.loc[train['date']=='2017-12-26'].sort_values('date')[['key','units']]
sales_prev6=train.loc[train['date']=='2017-12-25'].sort_values('date')[['key','units']]
sales_prev7=train.loc[train['date']=='2017-12-24'].sort_values('date')[['key','units']]
sales_prev8=train.loc[train['date']=='2017-12-23'].sort_values('date')[['key','units']]
sales_prev9=train.loc[train['date']=='2017-12-22'].sort_values('date')[['key','units']]
sales_prev10=train.loc[train['date']=='2017-12-21'].sort_values('date')[['key','units']]
sales_prev11=train.loc[train['date']=='2017-12-20'].sort_values('date')[['key','units']]

In [72]:
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-26'})
data_frame_2=data_frame_2.merge(sales_prev6, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-25'})
data_frame_2=data_frame_2.merge(sales_prev7, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-24'})
data_frame_2=data_frame_2.merge(sales_prev8, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-23'})
data_frame_2=data_frame_2.merge(sales_prev9, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-22'})
data_frame_2=data_frame_2.merge(sales_prev10, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-21'})
data_frame_2=data_frame_2.merge(sales_prev11, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-20'})

In [73]:
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')

In [75]:
c_train=train.merge(data_cluster,on='key',how='inner')

In [76]:
c_train.head()

,date,key,units,clust_7
0,2017-10-01,19671 39 1/3,0.0,1
1,2017-10-02,19671 39 1/3,0.0,1
2,2017-10-03,19671 39 1/3,0.0,1
3,2017-10-04,19671 39 1/3,0.0,1
4,2017-10-05,19671 39 1/3,1.0,1


In [77]:
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner')

In [79]:
## Running the clusters
def days_10_cluster(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    print(c_train_c1.shape)
    print(c_train_c1.key.nunique())
    train_comb_pivot=c_train_c1.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0) #assign zero sales
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=10, n_out=1, dropnan=True)
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose()
    model.fit(train_cluster1[['var1(t-10)', 'var1(t-9)', 'var1(t-8)', 'var1(t-7)', 'var1(t-6)',
                          'var1(t-5)', 'var1(t-4)', 'var1(t-3)', 'var1(t-2)', 'var1(t-1)']],train_cluster1[['var1(t)']])
    days=range(1,32)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=1)
        pred_date_before=predicted_date- timedelta(days=10)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=cluster1_frame[bet_day]
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    return(cluster1_frame)

In [80]:
regr = linear_model.LinearRegression()
cluster1_frame=days_10_cluster(c_dataframe,c_train,1,regr)

(806078, 3)
8858


In [81]:
regr = linear_model.LinearRegression()
cluster2_frame=days_10_cluster(c_dataframe,c_train,2,regr)

(156520, 3)
1720


In [82]:
regr = linear_model.LinearRegression()
cluster3_frame=days_10_cluster(c_dataframe,c_train,3,regr)

(25844, 3)
284


In [83]:
regr = linear_model.LinearRegression()
cluster4_frame=days_10_cluster(c_dataframe,c_train,4,regr)

(1001, 3)
11


In [84]:
regr = linear_model.LinearRegression()
cluster5_frame=days_10_cluster(c_dataframe,c_train,5,regr)

(8463, 3)
93


In [85]:
regr = linear_model.LinearRegression()
cluster6_frame=days_10_cluster(c_dataframe,c_train,6,regr)

(3640, 3)
40


In [86]:
regr = linear_model.LinearRegression()
cluster7_frame=days_10_cluster(c_dataframe,c_train,7,regr)

(364, 3)
4


## Performance Measuring

In [87]:
test_data=pd.read_csv('test_0.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39175        4668  8.392245
2.0               12634        1575  8.021587
3.0                1783         281  6.345196
4.0                  52          11  4.727273
5.0                 537          93  5.774194
6.0                 241          40  6.025000
7.0                   8           4  2.000000
8.0               10322        1470  7.021769


254.46414285710276

In [88]:
test_data=pd.read_csv('test_1.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39542        4668  8.470865
2.0               12803        1575  8.128889
3.0                1782         281  6.341637
4.0                  56          11  5.090909
5.0                 473          93  5.086022
6.0                 210          40  5.250000
7.0                  16           4  4.000000
8.0               10299        1470  7.006122


255.30569911382707

In [89]:
test_data=pd.read_csv('test_2.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39418        4668  8.444302
2.0               12754        1575  8.097778
3.0                1835         281  6.530249
4.0                  54          11  4.909091
5.0                 502          93  5.397849
6.0                 202          40  5.050000
7.0                  17           4  4.250000
8.0               10273        1470  6.988435


255.05881674625562

In [90]:
test_data=pd.read_csv('test_3.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39333        4668  8.426093
2.0               12857        1575  8.163175
3.0                1780         281  6.334520
4.0                  60          11  5.454545
5.0                 464          93  4.989247
6.0                 232          40  5.800000
7.0                  13           4  3.250000
8.0               10214        1470  6.948299


254.85878442776894

In [91]:
test_data=pd.read_csv('test_4.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39526        4668  8.467438
2.0               12876        1575  8.175238
3.0                1755         281  6.245552
4.0                  53          11  4.818182
5.0                 528          93  5.677419
6.0                 211          40  5.275000
7.0                  20           4  5.000000
8.0               10187        1470  6.929932


255.2567335057001

# 4. Windowing with clusters (Lag=15)

In [92]:
cluster7=pd.read_csv('clust_7.csv') #cluster number
data_cluster=pd.read_csv('data_clustering_R') #dataset used for clustering
data_cluster['clust_7']=cluster7['x'] #joining the clusters
data_cluster=data_cluster[['key','clust_7']]
d1 = date(2017, 10, 1)  # start date
d2 = date(2017, 12, 31)  # end date
d3=date(2018, 1, 31)
delta = d2 - d1         # timedelta

pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)

post_day=[] 
delta_2 = d3 - d2

for i in range(delta_2.days + 1):
    add=str(d2 + timedelta(days=i))
    post_day.append(add)
df_days_pre = pd.DataFrame({'date':pre_day})
df_days_post = pd.DataFrame({'date':post_day})
train=train_df[['date','key','units']]

In [93]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [94]:
sales_prev=train.loc[train['date']=='2017-12-31'].sort_values('date')[['key','units']]
sales_prev1=train.loc[train['date']=='2017-12-30'].sort_values('date')[['key','units']]
sales_prev2=train.loc[train['date']=='2017-12-29'].sort_values('date')[['key','units']]
sales_prev3=train.loc[train['date']=='2017-12-28'].sort_values('date')[['key','units']]
sales_prev4=train.loc[train['date']=='2017-12-27'].sort_values('date')[['key','units']]
sales_prev5=train.loc[train['date']=='2017-12-26'].sort_values('date')[['key','units']]
sales_prev6=train.loc[train['date']=='2017-12-25'].sort_values('date')[['key','units']]
sales_prev7=train.loc[train['date']=='2017-12-24'].sort_values('date')[['key','units']]
sales_prev8=train.loc[train['date']=='2017-12-23'].sort_values('date')[['key','units']]
sales_prev9=train.loc[train['date']=='2017-12-22'].sort_values('date')[['key','units']]
sales_prev10=train.loc[train['date']=='2017-12-21'].sort_values('date')[['key','units']]
sales_prev11=train.loc[train['date']=='2017-12-20'].sort_values('date')[['key','units']]
sales_prev12=train.loc[train['date']=='2017-12-19'].sort_values('date')[['key','units']]
sales_prev13=train.loc[train['date']=='2017-12-18'].sort_values('date')[['key','units']]
sales_prev14=train.loc[train['date']=='2017-12-17'].sort_values('date')[['key','units']]
sales_prev15=train.loc[train['date']=='2017-12-16'].sort_values('date')[['key','units']]
sales_prev16=train.loc[train['date']=='2017-12-15'].sort_values('date')[['key','units']]
sales_prev17=train.loc[train['date']=='2017-12-14'].sort_values('date')[['key','units']]

In [95]:
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-26'})
data_frame_2=data_frame_2.merge(sales_prev6, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-25'})
data_frame_2=data_frame_2.merge(sales_prev7, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-24'})
data_frame_2=data_frame_2.merge(sales_prev8, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-23'})
data_frame_2=data_frame_2.merge(sales_prev9, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-22'})
data_frame_2=data_frame_2.merge(sales_prev10, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-21'})
data_frame_2=data_frame_2.merge(sales_prev11, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-20'})
data_frame_2=data_frame_2.merge(sales_prev12, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-19'})
data_frame_2=data_frame_2.merge(sales_prev13, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-18'})
data_frame_2=data_frame_2.merge(sales_prev14, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-17'})
data_frame_2=data_frame_2.merge(sales_prev15, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-16'})
data_frame_2=data_frame_2.merge(sales_prev16, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-15'})
data_frame_2=data_frame_2.merge(sales_prev17, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-14'})

In [96]:
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')

In [97]:
c_train=train.merge(data_cluster,on='key',how='inner')

In [98]:
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner')

In [99]:
## Running the clusters
def days_15_cluster(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    print(c_train_c1.shape)
    print(c_train_c1.key.nunique())
    train_comb_pivot=c_train_c1.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0) #assign zero sales
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=15, n_out=1, dropnan=True)
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose()
    model.fit(train_cluster1[['var1(t-15)', 'var1(t-14)', 'var1(t-13)', 'var1(t-12)',
       'var1(t-11)', 'var1(t-10)', 'var1(t-9)', 'var1(t-8)', 'var1(t-7)',
       'var1(t-6)', 'var1(t-5)', 'var1(t-4)', 'var1(t-3)', 'var1(t-2)',
       'var1(t-1)']],train_cluster1[['var1(t)']])
    
    days=range(1,32)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=1)
        pred_date_before=predicted_date- timedelta(days=15)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=cluster1_frame[bet_day]
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    return(cluster1_frame)

In [100]:
regr = linear_model.LinearRegression()
cluster1_frame=days_15_cluster(c_dataframe,c_train,1,regr)

(806078, 3)
8858


In [101]:
regr = linear_model.LinearRegression()
cluster2_frame=days_15_cluster(c_dataframe,c_train,2,regr)

(156520, 3)
1720


In [102]:
regr = linear_model.LinearRegression()
cluster3_frame=days_15_cluster(c_dataframe,c_train,3,regr)

(25844, 3)
284


In [103]:
regr = linear_model.LinearRegression()
cluster4_frame=days_15_cluster(c_dataframe,c_train,4,regr)

(1001, 3)
11


In [104]:
regr = linear_model.LinearRegression()
cluster5_frame=days_15_cluster(c_dataframe,c_train,5,regr)

(8463, 3)
93


In [105]:
regr = linear_model.LinearRegression()
cluster6_frame=days_15_cluster(c_dataframe,c_train,6,regr)

(3640, 3)
40


In [106]:
regr = linear_model.LinearRegression()
cluster7_frame=days_15_cluster(c_dataframe,c_train,7,regr)

(364, 3)
4


## Performance Measure

In [107]:
test_data=pd.read_csv('test_0.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39175        4668  8.392245
2.0               12607        1575  8.004444
3.0                1775         281  6.316726
4.0                  54          11  4.909091
5.0                 532          93  5.720430
6.0                 237          40  5.925000
7.0                  10           4  2.500000
8.0               10322        1470  7.021769


254.3855341799136

In [108]:
test_data=pd.read_csv('test_1.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39542        4668  8.470865
2.0               12710        1575  8.069841
3.0                1749         281  6.224199
4.0                  55          11  5.000000
5.0                 466          93  5.010753
6.0                 210          40  5.250000
7.0                  20           4  5.000000
8.0               10299        1470  7.006122


255.05097529709624

In [109]:
test_data=pd.read_csv('test_2.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39418        4668  8.444302
2.0               12709        1575  8.069206
3.0                1833         281  6.523132
4.0                  53          11  4.818182
5.0                 500          93  5.376344
6.0                 202          40  5.050000
7.0                  21           4  5.250000
8.0               10273        1470  6.988435


254.9686255208668

In [110]:
test_data=pd.read_csv('test_3.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39333        4668  8.426093
2.0               12775        1575  8.111111
3.0                1764         281  6.277580
4.0                  58          11  5.272727
5.0                 454          93  4.881720
6.0                 227          40  5.675000
7.0                  16           4  4.000000
8.0               10214        1470  6.948299


254.6389600984107

In [111]:
test_data=pd.read_csv('test_4.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39526        4668  8.467438
2.0               12792        1575  8.121905
3.0                1740         281  6.192171
4.0                  53          11  4.818182
5.0                 524          93  5.634409
6.0                 216          40  5.400000
7.0                  22           4  5.500000
8.0               10187        1470  6.929932


255.068618218706

# 4.Windowing with clusters (Lag=20)

In [112]:
cluster7=pd.read_csv('clust_7.csv') #cluster number
data_cluster=pd.read_csv('data_clustering_R') #dataset used for clustering
data_cluster['clust_7']=cluster7['x'] #joining the clusters
data_cluster=data_cluster[['key','clust_7']]
d1 = date(2017, 10, 1)  # start date
d2 = date(2017, 12, 31)  # end date
d3=date(2018, 1, 31)
delta = d2 - d1         # timedelta

pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)

post_day=[] 
delta_2 = d3 - d2

for i in range(delta_2.days + 1):
    add=str(d2 + timedelta(days=i))
    post_day.append(add)
df_days_pre = pd.DataFrame({'date':pre_day})
df_days_post = pd.DataFrame({'date':post_day})
train=train_df[['date','key','units']]

In [113]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [114]:
sales_prev=train.loc[train['date']=='2017-12-31'].sort_values('date')[['key','units']]
sales_prev1=train.loc[train['date']=='2017-12-30'].sort_values('date')[['key','units']]
sales_prev2=train.loc[train['date']=='2017-12-29'].sort_values('date')[['key','units']]
sales_prev3=train.loc[train['date']=='2017-12-28'].sort_values('date')[['key','units']]
sales_prev4=train.loc[train['date']=='2017-12-27'].sort_values('date')[['key','units']]
sales_prev5=train.loc[train['date']=='2017-12-26'].sort_values('date')[['key','units']]
sales_prev6=train.loc[train['date']=='2017-12-25'].sort_values('date')[['key','units']]
sales_prev7=train.loc[train['date']=='2017-12-24'].sort_values('date')[['key','units']]
sales_prev8=train.loc[train['date']=='2017-12-23'].sort_values('date')[['key','units']] 
sales_prev9=train.loc[train['date']=='2017-12-22'].sort_values('date')[['key','units']]
sales_prev10=train.loc[train['date']=='2017-12-21'].sort_values('date')[['key','units']]
sales_prev11=train.loc[train['date']=='2017-12-20'].sort_values('date')[['key','units']]
sales_prev12=train.loc[train['date']=='2017-12-19'].sort_values('date')[['key','units']]
sales_prev13=train.loc[train['date']=='2017-12-18'].sort_values('date')[['key','units']]
sales_prev14=train.loc[train['date']=='2017-12-17'].sort_values('date')[['key','units']]
sales_prev15=train.loc[train['date']=='2017-12-16'].sort_values('date')[['key','units']]
sales_prev16=train.loc[train['date']=='2017-12-15'].sort_values('date')[['key','units']]
sales_prev17=train.loc[train['date']=='2017-12-14'].sort_values('date')[['key','units']]
sales_prev18=train.loc[train['date']=='2017-12-13'].sort_values('date')[['key','units']]
sales_prev19=train.loc[train['date']=='2017-12-12'].sort_values('date')[['key','units']]
sales_prev20=train.loc[train['date']=='2017-12-11'].sort_values('date')[['key','units']]
sales_prev21=train.loc[train['date']=='2017-12-10'].sort_values('date')[['key','units']]
sales_prev22=train.loc[train['date']=='2017-12-09'].sort_values('date')[['key','units']]
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-26'})
data_frame_2=data_frame_2.merge(sales_prev6, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-25'})
data_frame_2=data_frame_2.merge(sales_prev7, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-24'})
data_frame_2=data_frame_2.merge(sales_prev8, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-23'})
data_frame_2=data_frame_2.merge(sales_prev9, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-22'})
data_frame_2=data_frame_2.merge(sales_prev10, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-21'})
data_frame_2=data_frame_2.merge(sales_prev11, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-20'})
data_frame_2=data_frame_2.merge(sales_prev12, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-19'})
data_frame_2=data_frame_2.merge(sales_prev13, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-18'})
data_frame_2=data_frame_2.merge(sales_prev14, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-17'})
data_frame_2=data_frame_2.merge(sales_prev15, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-16'})
data_frame_2=data_frame_2.merge(sales_prev16, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-15'})
data_frame_2=data_frame_2.merge(sales_prev17, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-14'})
data_frame_2=data_frame_2.merge(sales_prev18, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-13'})
data_frame_2=data_frame_2.merge(sales_prev19, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-12'})
data_frame_2=data_frame_2.merge(sales_prev20, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-11'})
data_frame_2=data_frame_2.merge(sales_prev21, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-10'})
data_frame_2=data_frame_2.merge(sales_prev22, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-09'})

In [115]:
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')
c_train=train.merge(data_cluster,on='key',how='inner')
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner')

In [116]:
c_train.head()

,date,key,units,clust_7
0,2017-10-01,19671 39 1/3,0.0,1
1,2017-10-02,19671 39 1/3,0.0,1
2,2017-10-03,19671 39 1/3,0.0,1
3,2017-10-04,19671 39 1/3,0.0,1
4,2017-10-05,19671 39 1/3,1.0,1


In [119]:
## Running the clusters
def days_20_cluster(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    print(c_train_c1.shape)
    print(c_train_c1.key.nunique())
    train_comb_pivot=c_train_c1.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0) #assign zero sales
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=20, n_out=1, dropnan=True)
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose()
    model.fit(train_cluster1[['var1(t-20)', 'var1(t-19)', 'var1(t-18)', 'var1(t-17)',
       'var1(t-16)', 'var1(t-15)', 'var1(t-14)', 'var1(t-13)',
       'var1(t-12)', 'var1(t-11)', 'var1(t-10)', 'var1(t-9)', 'var1(t-8)',
       'var1(t-7)', 'var1(t-6)', 'var1(t-5)', 'var1(t-4)', 'var1(t-3)',
       'var1(t-2)', 'var1(t-1)']],train_cluster1[['var1(t)']])
    
    days=range(1,32)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=1)
        pred_date_before=predicted_date- timedelta(days=20)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=cluster1_frame[bet_day]
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    return(cluster1_frame)

In [125]:
regr = linear_model.LinearRegression()
cluster1_frame=days_20_cluster(c_dataframe,c_train,1,regr)

(806078, 3)
8858


In [126]:
regr = linear_model.LinearRegression()
cluster2_frame=days_20_cluster(c_dataframe,c_train,2,regr)

(156520, 3)
1720


In [127]:
regr = linear_model.LinearRegression()
cluster3_frame=days_20_cluster(c_dataframe,c_train,3,regr)

(25844, 3)
284


In [128]:
regr = linear_model.LinearRegression()
cluster4_frame=days_20_cluster(c_dataframe,c_train,4,regr)

(1001, 3)
11


In [129]:
regr = linear_model.LinearRegression()
cluster5_frame=days_20_cluster(c_dataframe,c_train,5,regr)

(8463, 3)
93


In [130]:
regr = linear_model.LinearRegression()
cluster6_frame=days_20_cluster(c_dataframe,c_train,6,regr)

(3640, 3)
40


In [131]:
regr = linear_model.LinearRegression()
cluster7_frame=days_20_cluster(c_dataframe,c_train,7,regr)

(364, 3)
4


## Performance Measuring

In [132]:
test_data=pd.read_csv('test_0.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               37577        4668  8.049914
2.0               12941        1575  8.216508
3.0                1879         281  6.686833
4.0                  63          11  5.727273
5.0                 566          93  6.086022
6.0                 230          40  5.750000
7.0                   9           4  2.250000
8.0               10322        1470  7.021769


252.16462876462273

In [133]:
test_data=pd.read_csv('test_1.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               37916        4668  8.122536
2.0               13071        1575  8.299048
3.0                1829         281  6.508897
4.0                  66          11  6.000000
5.0                 497          93  5.344086
6.0                 210          40  5.250000
7.0                  17           4  4.250000
8.0               10299        1470  7.006122


252.79438284898657

In [134]:
test_data=pd.read_csv('test_2.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               37816        4668  8.101114
2.0               13091        1575  8.311746
3.0                1915         281  6.814947
4.0                  65          11  5.909091
5.0                 520          93  5.591398
6.0                 199          40  4.975000
7.0                  19           4  4.750000
8.0               10273        1470  6.988435


252.78053722547548

In [135]:
test_data=pd.read_csv('test_3.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               37679        4668  8.071765
2.0               13160        1575  8.355556
3.0                1869         281  6.651246
4.0                  69          11  6.272727
5.0                 481          93  5.172043
6.0                 228          40  5.700000
7.0                  14           4  3.500000
8.0               10214        1470  6.948299


252.4163227685563

In [136]:
test_data=pd.read_csv('test_4.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               37882        4668  8.115253
2.0               13193        1575  8.376508
3.0                1831         281  6.516014
4.0                  64          11  5.818182
5.0                 554          93  5.956989
6.0                 209          40  5.225000
7.0                  21           4  5.250000
8.0               10187        1470  6.929932


252.86557693762904

# 5. Windowing with clusters (Lag=30)

In [151]:
cluster7=pd.read_csv('clust_7.csv') #cluster number
data_cluster=pd.read_csv('data_clustering_R') #dataset used for clustering
data_cluster['clust_7']=cluster7['x'] #joining the clusters
data_cluster=data_cluster[['key','clust_7']]
d1 = date(2017, 10, 1)  # start date
d2 = date(2017, 12, 31)  # end date
d3=date(2018, 1, 31)
delta = d2 - d1         # timedelta

pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)

post_day=[] 
delta_2 = d3 - d2

for i in range(delta_2.days + 1):
    add=str(d2 + timedelta(days=i))
    post_day.append(add)
df_days_pre = pd.DataFrame({'date':pre_day})
df_days_post = pd.DataFrame({'date':post_day})
train=train_df[['date','key','units']]

In [152]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [153]:
sales_prev=train.loc[train['date']=='2017-12-31'].sort_values('date')[['key','units']]
sales_prev1=train.loc[train['date']=='2017-12-30'].sort_values('date')[['key','units']]
sales_prev2=train.loc[train['date']=='2017-12-29'].sort_values('date')[['key','units']]
sales_prev3=train.loc[train['date']=='2017-12-28'].sort_values('date')[['key','units']]
sales_prev4=train.loc[train['date']=='2017-12-27'].sort_values('date')[['key','units']]
sales_prev5=train.loc[train['date']=='2017-12-26'].sort_values('date')[['key','units']]
sales_prev6=train.loc[train['date']=='2017-12-25'].sort_values('date')[['key','units']]
sales_prev7=train.loc[train['date']=='2017-12-24'].sort_values('date')[['key','units']]
sales_prev8=train.loc[train['date']=='2017-12-23'].sort_values('date')[['key','units']] 
sales_prev9=train.loc[train['date']=='2017-12-22'].sort_values('date')[['key','units']]
sales_prev10=train.loc[train['date']=='2017-12-21'].sort_values('date')[['key','units']]
sales_prev11=train.loc[train['date']=='2017-12-20'].sort_values('date')[['key','units']]
sales_prev12=train.loc[train['date']=='2017-12-19'].sort_values('date')[['key','units']]
sales_prev13=train.loc[train['date']=='2017-12-18'].sort_values('date')[['key','units']]
sales_prev14=train.loc[train['date']=='2017-12-17'].sort_values('date')[['key','units']]
sales_prev15=train.loc[train['date']=='2017-12-16'].sort_values('date')[['key','units']]
sales_prev16=train.loc[train['date']=='2017-12-15'].sort_values('date')[['key','units']]
sales_prev17=train.loc[train['date']=='2017-12-14'].sort_values('date')[['key','units']]
sales_prev18=train.loc[train['date']=='2017-12-13'].sort_values('date')[['key','units']]
sales_prev19=train.loc[train['date']=='2017-12-12'].sort_values('date')[['key','units']]
sales_prev20=train.loc[train['date']=='2017-12-11'].sort_values('date')[['key','units']]
sales_prev21=train.loc[train['date']=='2017-12-10'].sort_values('date')[['key','units']]
sales_prev22=train.loc[train['date']=='2017-12-09'].sort_values('date')[['key','units']]
sales_prev23=train.loc[train['date']=='2017-12-08'].sort_values('date')[['key','units']]
sales_prev24=train.loc[train['date']=='2017-12-07'].sort_values('date')[['key','units']]
sales_prev25=train.loc[train['date']=='2017-12-06'].sort_values('date')[['key','units']]
sales_prev26=train.loc[train['date']=='2017-12-05'].sort_values('date')[['key','units']]
sales_prev27=train.loc[train['date']=='2017-12-04'].sort_values('date')[['key','units']]
sales_prev28=train.loc[train['date']=='2017-12-03'].sort_values('date')[['key','units']]
sales_prev29=train.loc[train['date']=='2017-12-02'].sort_values('date')[['key','units']]
sales_prev30=train.loc[train['date']=='2017-12-01'].sort_values('date')[['key','units']]
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-26'})
data_frame_2=data_frame_2.merge(sales_prev6, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-25'})
data_frame_2=data_frame_2.merge(sales_prev7, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-24'})
data_frame_2=data_frame_2.merge(sales_prev8, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-23'})
data_frame_2=data_frame_2.merge(sales_prev9, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-22'})
data_frame_2=data_frame_2.merge(sales_prev10, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-21'})
data_frame_2=data_frame_2.merge(sales_prev11, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-20'})
data_frame_2=data_frame_2.merge(sales_prev12, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-19'})
data_frame_2=data_frame_2.merge(sales_prev13, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-18'})
data_frame_2=data_frame_2.merge(sales_prev14, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-17'})
data_frame_2=data_frame_2.merge(sales_prev15, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-16'})
data_frame_2=data_frame_2.merge(sales_prev16, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-15'})
data_frame_2=data_frame_2.merge(sales_prev17, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-14'})
data_frame_2=data_frame_2.merge(sales_prev18, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-13'})
data_frame_2=data_frame_2.merge(sales_prev19, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-12'})
data_frame_2=data_frame_2.merge(sales_prev20, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-11'})
data_frame_2=data_frame_2.merge(sales_prev21, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-10'})
data_frame_2=data_frame_2.merge(sales_prev22, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-09'})
data_frame_2=data_frame_2.merge(sales_prev23, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-08'})
data_frame_2=data_frame_2.merge(sales_prev24, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-07'})
data_frame_2=data_frame_2.merge(sales_prev25, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-06'})
data_frame_2=data_frame_2.merge(sales_prev26, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-05'})
data_frame_2=data_frame_2.merge(sales_prev27, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-04'})
data_frame_2=data_frame_2.merge(sales_prev28, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-03'})
data_frame_2=data_frame_2.merge(sales_prev29, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-02'})
data_frame_2=data_frame_2.merge(sales_prev30, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-01'})

In [154]:
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')
c_train=train.merge(data_cluster,on='key',how='inner')
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner')

In [155]:
## Running the clusters
def days_30_cluster(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    print(c_train_c1.shape)
    print(c_train_c1.key.nunique())
    train_comb_pivot=c_train_c1.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0) #assign zero sales
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=30, n_out=1, dropnan=True)
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose()
    model.fit(train_cluster1[['var1(t-30)', 'var1(t-29)', 'var1(t-28)', 'var1(t-27)',
       'var1(t-26)', 'var1(t-25)', 'var1(t-24)', 'var1(t-23)',
       'var1(t-22)', 'var1(t-21)', 'var1(t-20)', 'var1(t-19)',
       'var1(t-18)', 'var1(t-17)', 'var1(t-16)', 'var1(t-15)',
       'var1(t-14)', 'var1(t-13)', 'var1(t-12)', 'var1(t-11)',
       'var1(t-10)', 'var1(t-9)', 'var1(t-8)', 'var1(t-7)', 'var1(t-6)',
       'var1(t-5)', 'var1(t-4)', 'var1(t-3)', 'var1(t-2)', 'var1(t-1)']],train_cluster1[['var1(t)']])
    
    days=range(1,32)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=1)
        pred_date_before=predicted_date- timedelta(days=30)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=cluster1_frame[bet_day]
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    return(cluster1_frame)

In [161]:
regr = linear_model.LinearRegression()
cluster1_frame=days_30_cluster(c_dataframe,c_train,1,regr)

(806078, 3)
8858


In [162]:
regr = linear_model.LinearRegression()
cluster2_frame=days_30_cluster(c_dataframe,c_train,2,regr)

(156520, 3)
1720


In [163]:
regr = linear_model.LinearRegression()
cluster3_frame=days_30_cluster(c_dataframe,c_train,3,regr)

(25844, 3)
284


In [164]:
regr = linear_model.LinearRegression()
cluster4_frame=days_30_cluster(c_dataframe,c_train,4,regr)

(1001, 3)
11


In [165]:
regr = linear_model.LinearRegression()
cluster5_frame=days_30_cluster(c_dataframe,c_train,5,regr)

(8463, 3)
93


In [166]:
regr = linear_model.LinearRegression()
cluster6_frame=days_30_cluster(c_dataframe,c_train,6,regr)

(3640, 3)
40


In [167]:
regr = linear_model.LinearRegression()
cluster7_frame=days_30_cluster(c_dataframe,c_train,7,regr)

(364, 3)
4


## Performance Measurement

In [168]:
test_data=pd.read_csv('test_0.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39175        4668  8.392245
2.0               11513        1575  7.309841
3.0                1797         281  6.395018
4.0                  67          11  6.090909
5.0                 571          93  6.139785
6.0                 234          40  5.850000
7.0                   9           4  2.250000
8.0               10322        1470  7.021769


252.36481529722008

In [169]:
test_data=pd.read_csv('test_1.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39542        4668  8.470865
2.0               11601        1575  7.365714
3.0                1714         281  6.099644
4.0                  68          11  6.181818
5.0                 500          93  5.376344
6.0                 221          40  5.525000
7.0                  17           4  4.250000
8.0               10299        1470  7.006122


252.90709756746645

In [170]:
test_data=pd.read_csv('test_2.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39418        4668  8.444302
2.0               11715        1575  7.438095
3.0                1814         281  6.455516
4.0                  66          11  6.000000
5.0                 524          93  5.634409
6.0                 213          40  5.325000
7.0                  19           4  4.750000
8.0               10273        1470  6.988435


253.06520898772317

In [171]:
test_data=pd.read_csv('test_3.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39333        4668  8.426093
2.0               11736        1575  7.451429
3.0                1750         281  6.227758
4.0                  72          11  6.545455
5.0                 480          93  5.161290
6.0                 228          40  5.700000
7.0                  13           4  3.250000
8.0               10214        1470  6.948299


252.6380810566768

In [172]:
test_data=pd.read_csv('test_4.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39526        4668  8.467438
2.0               11756        1575  7.464127
3.0                1697         281  6.039146
4.0                  66          11  6.000000
5.0                 555          93  5.967742
6.0                 222          40  5.550000
7.0                  17           4  4.250000
8.0               10187        1470  6.929932


253.03359460751452

# 6. Windowing with Clusters (Lag=40)

In [173]:
cluster7=pd.read_csv('clust_7.csv') #cluster number
data_cluster=pd.read_csv('data_clustering_R') #dataset used for clustering
data_cluster['clust_7']=cluster7['x'] #joining the clusters
data_cluster=data_cluster[['key','clust_7']]
d1 = date(2017, 10, 1)  # start date
d2 = date(2017, 12, 31)  # end date
d3=date(2018, 1, 31)
delta = d2 - d1         # timedelta

pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)

post_day=[] 
delta_2 = d3 - d2

for i in range(delta_2.days + 1):
    add=str(d2 + timedelta(days=i))
    post_day.append(add)
df_days_pre = pd.DataFrame({'date':pre_day})
df_days_post = pd.DataFrame({'date':post_day})
train=train_df[['date','key','units']]

In [174]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [175]:
sales_prev=train.loc[train['date']=='2017-12-31'].sort_values('date')[['key','units']]
sales_prev1=train.loc[train['date']=='2017-12-30'].sort_values('date')[['key','units']]
sales_prev2=train.loc[train['date']=='2017-12-29'].sort_values('date')[['key','units']]
sales_prev3=train.loc[train['date']=='2017-12-28'].sort_values('date')[['key','units']]
sales_prev4=train.loc[train['date']=='2017-12-27'].sort_values('date')[['key','units']]
sales_prev5=train.loc[train['date']=='2017-12-26'].sort_values('date')[['key','units']]
sales_prev6=train.loc[train['date']=='2017-12-25'].sort_values('date')[['key','units']]
sales_prev7=train.loc[train['date']=='2017-12-24'].sort_values('date')[['key','units']]
sales_prev8=train.loc[train['date']=='2017-12-23'].sort_values('date')[['key','units']] 
sales_prev9=train.loc[train['date']=='2017-12-22'].sort_values('date')[['key','units']]
sales_prev10=train.loc[train['date']=='2017-12-21'].sort_values('date')[['key','units']]
sales_prev11=train.loc[train['date']=='2017-12-20'].sort_values('date')[['key','units']]
sales_prev12=train.loc[train['date']=='2017-12-19'].sort_values('date')[['key','units']]
sales_prev13=train.loc[train['date']=='2017-12-18'].sort_values('date')[['key','units']]
sales_prev14=train.loc[train['date']=='2017-12-17'].sort_values('date')[['key','units']]
sales_prev15=train.loc[train['date']=='2017-12-16'].sort_values('date')[['key','units']]
sales_prev16=train.loc[train['date']=='2017-12-15'].sort_values('date')[['key','units']]
sales_prev17=train.loc[train['date']=='2017-12-14'].sort_values('date')[['key','units']]
sales_prev18=train.loc[train['date']=='2017-12-13'].sort_values('date')[['key','units']]
sales_prev19=train.loc[train['date']=='2017-12-12'].sort_values('date')[['key','units']]
sales_prev20=train.loc[train['date']=='2017-12-11'].sort_values('date')[['key','units']]
sales_prev21=train.loc[train['date']=='2017-12-10'].sort_values('date')[['key','units']]
sales_prev22=train.loc[train['date']=='2017-12-09'].sort_values('date')[['key','units']]
sales_prev23=train.loc[train['date']=='2017-12-08'].sort_values('date')[['key','units']]
sales_prev24=train.loc[train['date']=='2017-12-07'].sort_values('date')[['key','units']]
sales_prev25=train.loc[train['date']=='2017-12-06'].sort_values('date')[['key','units']]
sales_prev26=train.loc[train['date']=='2017-12-05'].sort_values('date')[['key','units']]
sales_prev27=train.loc[train['date']=='2017-12-04'].sort_values('date')[['key','units']]
sales_prev28=train.loc[train['date']=='2017-12-03'].sort_values('date')[['key','units']]
sales_prev29=train.loc[train['date']=='2017-12-02'].sort_values('date')[['key','units']]
sales_prev30=train.loc[train['date']=='2017-12-01'].sort_values('date')[['key','units']]
sales_prev31=train.loc[train['date']=='2017-11-30'].sort_values('date')[['key','units']]
sales_prev32=train.loc[train['date']=='2017-11-29'].sort_values('date')[['key','units']]
sales_prev33=train.loc[train['date']=='2017-11-28'].sort_values('date')[['key','units']]
sales_prev34=train.loc[train['date']=='2017-11-27'].sort_values('date')[['key','units']]
sales_prev35=train.loc[train['date']=='2017-11-26'].sort_values('date')[['key','units']]
sales_prev36=train.loc[train['date']=='2017-11-25'].sort_values('date')[['key','units']]
sales_prev37=train.loc[train['date']=='2017-11-24'].sort_values('date')[['key','units']]
sales_prev38=train.loc[train['date']=='2017-11-23'].sort_values('date')[['key','units']]
sales_prev39=train.loc[train['date']=='2017-11-22'].sort_values('date')[['key','units']]
sales_prev40=train.loc[train['date']=='2017-11-21'].sort_values('date')[['key','units']]
sales_prev41=train.loc[train['date']=='2017-11-20'].sort_values('date')[['key','units']]
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-26'})
data_frame_2=data_frame_2.merge(sales_prev6, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-25'})
data_frame_2=data_frame_2.merge(sales_prev7, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-24'})
data_frame_2=data_frame_2.merge(sales_prev8, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-23'})
data_frame_2=data_frame_2.merge(sales_prev9, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-22'})
data_frame_2=data_frame_2.merge(sales_prev10, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-21'})
data_frame_2=data_frame_2.merge(sales_prev11, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-20'})
data_frame_2=data_frame_2.merge(sales_prev12, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-19'})
data_frame_2=data_frame_2.merge(sales_prev13, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-18'})
data_frame_2=data_frame_2.merge(sales_prev14, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-17'})
data_frame_2=data_frame_2.merge(sales_prev15, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-16'})
data_frame_2=data_frame_2.merge(sales_prev16, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-15'})
data_frame_2=data_frame_2.merge(sales_prev17, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-14'})
data_frame_2=data_frame_2.merge(sales_prev18, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-13'})
data_frame_2=data_frame_2.merge(sales_prev19, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-12'})
data_frame_2=data_frame_2.merge(sales_prev20, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-11'})
data_frame_2=data_frame_2.merge(sales_prev21, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-10'})
data_frame_2=data_frame_2.merge(sales_prev22, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-09'})
data_frame_2=data_frame_2.merge(sales_prev23, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-08'})
data_frame_2=data_frame_2.merge(sales_prev24, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-07'})
data_frame_2=data_frame_2.merge(sales_prev25, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-06'})
data_frame_2=data_frame_2.merge(sales_prev26, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-05'})
data_frame_2=data_frame_2.merge(sales_prev27, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-04'})
data_frame_2=data_frame_2.merge(sales_prev28, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-03'})
data_frame_2=data_frame_2.merge(sales_prev29, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-02'})
data_frame_2=data_frame_2.merge(sales_prev30, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-01'})
data_frame_2=data_frame_2.merge(sales_prev31, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-30'})
data_frame_2=data_frame_2.merge(sales_prev32, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-29'})
data_frame_2=data_frame_2.merge(sales_prev33, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-28'})
data_frame_2=data_frame_2.merge(sales_prev34, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-27'})
data_frame_2=data_frame_2.merge(sales_prev35, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-26'})
data_frame_2=data_frame_2.merge(sales_prev36, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-25'})
data_frame_2=data_frame_2.merge(sales_prev37, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-24'})
data_frame_2=data_frame_2.merge(sales_prev38, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-23'})
data_frame_2=data_frame_2.merge(sales_prev39, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-22'})
data_frame_2=data_frame_2.merge(sales_prev40, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-21'})
data_frame_2=data_frame_2.merge(sales_prev41, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-11-20'})

In [176]:
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')
c_train=train.merge(data_cluster,on='key',how='inner')
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner')

In [177]:
## Running the clusters
def days_40_cluster(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    print(c_train_c1.shape)
    print(c_train_c1.key.nunique())
    train_comb_pivot=c_train_c1.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0) #assign zero sales
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=40, n_out=1, dropnan=True)
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose()
    model.fit(train_cluster1[['var1(t-40)','var1(t-39)','var1(t-38)','var1(t-37)','var1(t-36)','var1(t-35)','var1(t-34)',
                         'var1(t-33)','var1(t-32)','var1(t-31)','var1(t-30)', 'var1(t-29)', 'var1(t-28)', 'var1(t-27)',
       'var1(t-26)', 'var1(t-25)', 'var1(t-24)', 'var1(t-23)',
       'var1(t-22)', 'var1(t-21)', 'var1(t-20)', 'var1(t-19)',
       'var1(t-18)', 'var1(t-17)', 'var1(t-16)', 'var1(t-15)',
       'var1(t-14)', 'var1(t-13)', 'var1(t-12)', 'var1(t-11)',
       'var1(t-10)', 'var1(t-9)', 'var1(t-8)', 'var1(t-7)', 'var1(t-6)',
           'var1(t-5)', 'var1(t-4)', 'var1(t-3)', 'var1(t-2)', 'var1(t-1)']],train_cluster1[['var1(t)']])
    
    days=range(1,32)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=1)
        pred_date_before=predicted_date- timedelta(days=40)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=cluster1_frame[bet_day]
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    return(cluster1_frame)

In [182]:
regr = linear_model.LinearRegression()
cluster1_frame=days_40_cluster(c_dataframe,c_train,1,regr)

(806078, 3)
8858


In [183]:
regr = linear_model.LinearRegression()
cluster2_frame=days_40_cluster(c_dataframe,c_train,2,regr)

(156520, 3)
1720


In [184]:
regr = linear_model.LinearRegression()
cluster3_frame=days_40_cluster(c_dataframe,c_train,3,regr)

(25844, 3)
284


In [185]:
regr = linear_model.LinearRegression()
cluster4_frame=days_40_cluster(c_dataframe,c_train,4,regr)

(1001, 3)
11


In [186]:
regr = linear_model.LinearRegression()
cluster5_frame=days_40_cluster(c_dataframe,c_train,5,regr)

(8463, 3)
93


In [187]:
regr = linear_model.LinearRegression()
cluster6_frame=days_40_cluster(c_dataframe,c_train,6,regr)

(3640, 3)
40


In [188]:
regr = linear_model.LinearRegression()
cluster7_frame=days_40_cluster(c_dataframe,c_train,7,regr)

(364, 3)
4


## Performance

In [189]:
test_data=pd.read_csv('test_0.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size       mean
clust_7                                       
1.0               51773        4668  11.091045
2.0               14380        1575   9.130159
3.0                3090         281  10.996441
4.0                 125          11  11.363636
5.0                1186          93  12.752688
6.0                 394          40   9.850000
7.0                  45           4  11.250000
8.0               10322        1470   7.021769


285.1578510229028

In [190]:
test_data=pd.read_csv('test_1.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size       mean
clust_7                                       
1.0               52082        4668  11.157241
2.0               14565        1575   9.247619
3.0                3034         281  10.797153
4.0                 126          11  11.454545
5.0                1271          93  13.666667
6.0                 429          40  10.725000
7.0                  35           4   8.750000
8.0               10299        1470   7.006122


286.0786605114055

In [191]:
test_data=pd.read_csv('test_2.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size       mean
clust_7                                       
1.0               51778        4668  11.092117
2.0               14747        1575   9.363175
3.0                3121         281  11.106762
4.0                 120          11  10.909091
5.0                1127          93  12.118280
6.0                 400          40  10.000000
7.0                  37           4   9.250000
8.0               10273        1470   6.988435


285.66238814376663

In [192]:
test_data=pd.read_csv('test_3.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size       mean
clust_7                                       
1.0               52003        4668  11.140317
2.0               14665        1575   9.311111
3.0                3053         281  10.864769
4.0                 126          11  11.454545
5.0                1299          93  13.967742
6.0                 472          40  11.800000
7.0                  40           4  10.000000
8.0               10214        1470   6.948299


286.1328362841287

In [193]:
test_data=pd.read_csv('test_4.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size       mean
clust_7                                       
1.0               52280        4668  11.199657
2.0               14735        1575   9.355556
3.0                3024         281  10.761566
4.0                 122          11  11.090909
5.0                1137          93  12.225806
6.0                 442          40  11.050000
7.0                  35           4   8.750000
8.0               10187        1470   6.929932


286.29006269865533

# 7. Windowing with clusters (Lag=10, Day_of_theweek, holiday, company_offers, weather)

In [251]:
cluster7=pd.read_csv('clust_7.csv') #cluster number
data_cluster=pd.read_csv('data_clustering_R') #dataset used for clustering
data_cluster['clust_7']=cluster7['x'] #joining the clusters
data_cluster=data_cluster[['key','clust_7']]
d1 = date(2017, 10, 1)  # start date
d2 = date(2017, 12, 31)  # end date
d3=date(2018, 1, 31)
delta = d2 - d1         # timedelta

pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)

post_day=[] 
delta_2 = d3 - d2

for i in range(delta_2.days + 1):
    add=str(d2 + timedelta(days=i))
    post_day.append(add)
df_days_pre = pd.DataFrame({'date':pre_day})
df_days_post = pd.DataFrame({'date':post_day})

In [253]:
train_df.shape

(1166984, 35)

In [254]:
train_df.head()

,Unnamed: 0,key,weekday,day,month,date,rrp,new size_L,new size_M,new size_S,...,new size_44,new size_43,units,avg_temp,median_temp,company_offer,holiday,sum_unit,price_daily_change,new_product
0,0,19671 39 1/3,6,1,10,2017-10-01,190.43,0,0,0,...,0,0,0.0,12.5625,12.50,0,0,0.0,0.0,0
1,1,19671 39 1/3,0,2,10,2017-10-02,190.43,0,0,0,...,0,0,0.0,13.3125,13.75,0,0,0.0,0.0,0
2,2,19671 39 1/3,1,3,10,2017-10-03,190.43,0,0,0,...,0,0,0.0,12.1875,12.50,0,1,0.0,0.0,0
3,3,19671 39 1/3,2,4,10,2017-10-04,190.43,0,0,0,...,0,0,0.0,10.7500,10.75,0,0,0.0,0.0,0
4,4,19671 39 1/3,3,5,10,2017-10-05,190.43,0,0,0,...,0,0,1.0,11.7500,11.50,0,0,1.0,0.0,0


In [256]:
train=train_df[['date','median_temp','weekday','holiday','company_offer','key','units']]
#train['date']=train['date'].dt.strftime('%Y-%m-%d')

In [257]:
train.head()

,date,median_temp,weekday,holiday,company_offer,key,units
0,2017-10-01,12.50,6,0,0,19671 39 1/3,0.0
1,2017-10-02,13.75,0,0,0,19671 39 1/3,0.0
2,2017-10-03,12.50,1,1,0,19671 39 1/3,0.0
3,2017-10-04,10.75,2,0,0,19671 39 1/3,0.0
4,2017-10-05,11.50,3,0,0,19671 39 1/3,1.0


In [258]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [259]:
sales_prev=train.loc[train['date']=='2017-12-31'].sort_values('date')[['key','units']]
sales_prev1=train.loc[train['date']=='2017-12-30'].sort_values('date')[['key','units']]
sales_prev2=train.loc[train['date']=='2017-12-29'].sort_values('date')[['key','units']]
sales_prev3=train.loc[train['date']=='2017-12-28'].sort_values('date')[['key','units']]
sales_prev4=train.loc[train['date']=='2017-12-27'].sort_values('date')[['key','units']]
sales_prev5=train.loc[train['date']=='2017-12-26'].sort_values('date')[['key','units']]
sales_prev6=train.loc[train['date']=='2017-12-25'].sort_values('date')[['key','units']]
sales_prev7=train.loc[train['date']=='2017-12-24'].sort_values('date')[['key','units']]
sales_prev8=train.loc[train['date']=='2017-12-23'].sort_values('date')[['key','units']]
sales_prev9=train.loc[train['date']=='2017-12-22'].sort_values('date')[['key','units']]
sales_prev10=train.loc[train['date']=='2017-12-21'].sort_values('date')[['key','units']]
sales_prev11=train.loc[train['date']=='2017-12-20'].sort_values('date')[['key','units']]
sales_prev12=train.loc[train['date']=='2017-12-19'].sort_values('date')[['key','units']]

In [260]:
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-26'})
data_frame_2=data_frame_2.merge(sales_prev6, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-25'})
data_frame_2=data_frame_2.merge(sales_prev7, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-24'})
data_frame_2=data_frame_2.merge(sales_prev8, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-23'})
data_frame_2=data_frame_2.merge(sales_prev9, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-22'})
data_frame_2=data_frame_2.merge(sales_prev10, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-21'})
data_frame_2=data_frame_2.merge(sales_prev11, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-20'})
data_frame_2=data_frame_2.merge(sales_prev12, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-19'})

In [261]:
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')

In [262]:
c_train=train.merge(data_cluster,on='key',how='inner') #for traning data

In [263]:
c_train.head()

,date,median_temp,weekday,holiday,company_offer,key,units,clust_7
0,2017-10-01,12.50,6,0,0,19671 39 1/3,0.0,1
1,2017-10-02,13.75,0,0,0,19671 39 1/3,0.0,1
2,2017-10-03,12.50,1,1,0,19671 39 1/3,0.0,1
3,2017-10-04,10.75,2,0,0,19671 39 1/3,0.0,1
4,2017-10-05,11.50,3,0,0,19671 39 1/3,1.0,1


In [265]:
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner')

In [266]:
c_dataframe.head() #this is going to be used for predictive analysis

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2017-12-27,2017-12-26,2017-12-25,2017-12-24,2017-12-23,2017-12-22,2017-12-21,2017-12-20,2017-12-19,clust_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


Weather Data Preperation:

In [267]:
weather_train=train[['date','median_temp']]
weather_train=weather_train.drop_duplicates()
weather_train=weather_train.sort_values('date')
weather_train=weather_train.reset_index().drop(['index'],axis=1)

In [268]:
weather_test=test_df[['date','median_temp']]
weather_test=weather_test.drop_duplicates()
weather_test=weather_test.sort_values('date')
weather_test=weather_test.reset_index().drop(['index'],axis=1)

In [269]:
all_weather=weather_train.append(weather_test).set_index('date').transpose()

Day of the week:

In [270]:
day_week_train=train[['date','weekday']]
day_week_train=day_week_train.drop_duplicates()
day_week_train=day_week_train.sort_values('date')
day_week_train=day_week_train.reset_index().drop(['index'],axis=1)

In [271]:
day_week_test=test_df[['date','weekday']]
day_week_test=day_week_test.drop_duplicates()
day_week_test=day_week_test.sort_values('date')
day_week_test=day_week_test.reset_index().drop(['index'],axis=1)

In [272]:
all_day_week=day_week_train.append(day_week_test).set_index('date').transpose()

Holiday:

In [273]:
holi_train=train[['date','holiday']]
holi_train=holi_train.drop_duplicates()
holi_train=holi_train.sort_values('date')
holi_train=holi_train.reset_index().drop(['index'],axis=1)

In [274]:
holi_test=test_df[['date','holiday']]
holi_test=holi_test.drop_duplicates()
holi_test=holi_test.sort_values('date')
holi_test=holi_test.reset_index().drop(['index'],axis=1)

In [275]:
all_holiday=holi_train.append(holi_test).set_index('date').transpose()

Company Offers

In [276]:
co_train=train[['date','company_offer']]
co_train=co_train.drop_duplicates()
co_train=co_train.sort_values('date')
co_train=co_train.reset_index().drop(['index'],axis=1)

In [277]:
co_test=test_df[['date','company_offer']]
co_test=co_test.drop_duplicates()
co_test=co_test.sort_values('date')
co_test=co_test.reset_index().drop(['index'],axis=1)

In [278]:
all_co=co_train.append(co_test).set_index('date').transpose()

In [301]:
## Running the clusters
def days_10_cluster_var(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    all_=c_train_c1.key.unique()
    n=len(c_train_c1.key.unique())
    i=0
    k=0
    n1=len(pre_day)
    days=[]
    ids=[]

    while(i<n):
        b=all_[i]
        k=0
        while(k<n1):
            a=pre_day[k] #day
            ids.append(b) #id
            days.append(a)
            k=k+1
        i=i+1
        
    train_days = pd.DataFrame({'date':days,'key':ids})
    train_comb=train_days.merge(c_train_c1[['date','key','units']],on=['date','key'],how='outer')
    train_comb=train_comb.fillna(0)
    print(train_comb.shape)
    print(len(all_))
    print(n1)
    print(n1*len(all_))
    
    train_comb_pivot=train_comb.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0)
    train_comb_pivot=train_comb_pivot.drop(['2017-11-24'])
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_comb_pivot['temp']=weather_train['median_temp'].values
    train_comb_pivot['week_day']=day_week_train['weekday'].values
    train_comb_pivot['company_offers']=co_train['company_offer'].values
    train_comb_pivot['holiday']=holi_train['holiday'].values
    
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=10, n_out=1, dropnan=True) #mean_sales,temp,week_day,company_offers,holiday
    
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose() #where the predictive data will be written
    
    model.fit(train_cluster1[['var1(t-10)', 'var2(t-10)', 'var3(t-10)', 'var4(t-10)',
       'var5(t-10)', 'var1(t-9)', 'var2(t-9)', 'var3(t-9)', 'var4(t-9)',
       'var5(t-9)', 'var1(t-8)', 'var2(t-8)', 'var3(t-8)', 'var4(t-8)',
       'var5(t-8)', 'var1(t-7)', 'var2(t-7)', 'var3(t-7)', 'var4(t-7)',
       'var5(t-7)', 'var1(t-6)', 'var2(t-6)', 'var3(t-6)', 'var4(t-6)',
       'var5(t-6)', 'var1(t-5)', 'var2(t-5)', 'var3(t-5)', 'var4(t-5)',
       'var5(t-5)', 'var1(t-4)', 'var2(t-4)', 'var3(t-4)', 'var4(t-4)',
       'var5(t-4)', 'var1(t-3)', 'var2(t-3)', 'var3(t-3)', 'var4(t-3)',
       'var5(t-3)', 'var1(t-2)', 'var2(t-2)', 'var3(t-2)', 'var4(t-2)',
       'var5(t-2)', 'var1(t-1)', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)',
       'var5(t-1)']],train_cluster1[['var1(t)']])
    
    days=range(1,32)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=1)
        pred_date_before=predicted_date- timedelta(days=10)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=pd.DataFrame([cluster1_frame[bet_day[0]].values, all_weather[bet_day[0]].values,all_day_week[bet_day[0]].values,all_holiday[bet_day[0]].values, all_co[bet_day[0]].values,
        cluster1_frame[bet_day[1]].values, all_weather[bet_day[1]].values,all_day_week[bet_day[1]].values,all_holiday[bet_day[1]].values, all_co[bet_day[1]].values,
        cluster1_frame[bet_day[2]].values, all_weather[bet_day[2]].values,all_day_week[bet_day[2]].values,all_holiday[bet_day[2]].values, all_co[bet_day[2]].values,
        cluster1_frame[bet_day[3]].values, all_weather[bet_day[3]].values,all_day_week[bet_day[3]].values,all_holiday[bet_day[3]].values, all_co[bet_day[3]].values,
        cluster1_frame[bet_day[4]].values, all_weather[bet_day[4]].values,all_day_week[bet_day[4]].values,all_holiday[bet_day[4]].values, all_co[bet_day[4]].values,
        cluster1_frame[bet_day[5]].values, all_weather[bet_day[5]].values,all_day_week[bet_day[5]].values,all_holiday[bet_day[5]].values, all_co[bet_day[5]].values,
        cluster1_frame[bet_day[6]].values, all_weather[bet_day[6]].values,all_day_week[bet_day[6]].values,all_holiday[bet_day[6]].values, all_co[bet_day[6]].values,
        cluster1_frame[bet_day[7]].values, all_weather[bet_day[7]].values,all_day_week[bet_day[7]].values,all_holiday[bet_day[7]].values, all_co[bet_day[7]].values,
        cluster1_frame[bet_day[8]].values, all_weather[bet_day[8]].values,all_day_week[bet_day[8]].values,all_holiday[bet_day[8]].values, all_co[bet_day[8]].values,
        cluster1_frame[bet_day[9]].values, all_weather[bet_day[9]].values,all_day_week[bet_day[9]].values,all_holiday[bet_day[9]].values, all_co[bet_day[9]].values])
        value_predict=value_predict.transpose()
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    
    return(cluster1_frame)

In [302]:
regr = linear_model.LinearRegression()
cluster1_frame=days_10_cluster_var(c_dataframe,c_train,1,regr)

(814936, 3)
8858
92
814936


In [303]:
regr = linear_model.LinearRegression()
cluster2_frame=days_10_cluster_var(c_dataframe,c_train,2,regr)

(158240, 3)
1720
92
158240


In [304]:
regr = linear_model.LinearRegression()
cluster3_frame=days_10_cluster_var(c_dataframe,c_train,3,regr)

(26128, 3)
284
92
26128


In [305]:
regr = linear_model.LinearRegression()
cluster4_frame=days_10_cluster_var(c_dataframe,c_train,4,regr)

(1012, 3)
11
92
1012


In [306]:
regr = linear_model.LinearRegression()
cluster5_frame=days_10_cluster_var(c_dataframe,c_train,5,regr)

(8556, 3)
93
92
8556


In [307]:
regr = linear_model.LinearRegression()
cluster6_frame=days_10_cluster_var(c_dataframe,c_train,6,regr)

(3680, 3)
40
92
3680


In [308]:
regr = linear_model.LinearRegression()
cluster7_frame=days_10_cluster_var(c_dataframe,c_train,7,regr)

(368, 3)
4
92
368


### Performance Measurement

In [309]:
test_data=pd.read_csv('test_0.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size       mean
clust_7                                       
1.0               43237        4668   9.262425
2.0               15142        1575   9.613968
3.0                2342         281   8.334520
4.0                 120          11  10.909091
5.0                 601          93   6.462366
6.0                 251          40   6.275000
7.0                  14           4   3.500000
8.0               10322        1470   7.021769


268.382190169169

In [310]:
test_data=pd.read_csv('test_1.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size       mean
clust_7                                       
1.0               42834        4668   9.176093
2.0               15360        1575   9.752381
3.0                2254         281   8.021352
4.0                 122          11  11.090909
5.0                 550          93   5.913978
6.0                 232          40   5.800000
7.0                  23           4   5.750000
8.0               10299        1470   7.006122


267.7200029881966

In [311]:
test_data=pd.read_csv('test_2.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size       mean
clust_7                                       
1.0               43036        4668   9.219366
2.0               15496        1575   9.838730
3.0                2381         281   8.473310
4.0                 117          11  10.636364
5.0                 569          93   6.118280
6.0                 220          40   5.500000
7.0                  25           4   6.250000
8.0               10273        1470   6.988435


268.5460854304155

In [312]:
test_data=pd.read_csv('test_3.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size       mean
clust_7                                       
1.0               42529        4668   9.110754
2.0               15379        1575   9.764444
3.0                2309         281   8.217082
4.0                 124          11  11.272727
5.0                 530          93   5.698925
6.0                 244          40   6.100000
7.0                  20           4   5.000000
8.0               10214        1470   6.948299


267.11233591880404

In [313]:
test_data=pd.read_csv('test_4.csv')
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size       mean
clust_7                                       
1.0               43022        4668   9.216367
2.0               15439        1575   9.802540
3.0                2279         281   8.110320
4.0                 119          11  10.818182
5.0                 584          93   6.279570
6.0                 232          40   5.800000
7.0                  28           4   7.000000
8.0               10187        1470   6.929932


268.12310605391696